In [1]:
# Import necessary libraries
from dash import dash_table, dcc, html
import dash
from dash.dependencies import Input, Output
from animal_shelter import AnimalShelter
import plotly.express as px
from threading import Thread
import base64

# Initialize the Dash app
app = dash.Dash(__name__)

# MongoDB CRUD operations class
shelter = AnimalShelter()

# Load initial data from MongoDB
data = shelter.read({})
if data:
    columns = [{"name": col, "id": col} for col in data[0].keys()]
else:
    columns = []

# Encode the logo image file
image_filename = 'grazioso_salvare_logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode('ascii')

# Layout of the Dash app
app.layout = html.Div([
    html.Div([
        html.Img(src='data:image/png;base64,{}'.format(encoded_image), style={'height':'10%', 'width':'10%'}),
        html.H1("Grazioso Salvare Animal Shelter Dashboard"),
    ], style={'display': 'flex', 'align-items': 'center'}),
    dcc.RadioItems(
        id='rescue-type-filter',
        options=[
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain Rescue', 'value': 'Mountain Rescue'},
            {'label': 'Disaster Rescue', 'value': 'Disaster Rescue'},
            {'label': 'Tracking', 'value': 'Tracking'}
        ],
        value='Water Rescue'
    ),
    dash_table.DataTable(
        id='datatable',
        columns=columns,
        data=data,
        page_size=10,
    ),
    dcc.Graph(id='geo-chart'),
    dcc.Graph(id='additional-chart')
])

@app.callback(
    [Output('datatable', 'data'),
     Output('geo-chart', 'figure'),
     Output('additional-chart', 'figure')],
    [Input('rescue-type-filter', 'value')]
)
def update_dashboard(selected_rescue):
    # Filter data based on rescue type
    if selected_rescue == 'Water Rescue':
        filtered_data = shelter.read({"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}})
    elif selected_rescue == 'Mountain Rescue':
        filtered_data = shelter.read({"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}})
    elif selected_rescue == 'Disaster Rescue':
        filtered_data = shelter.read({"breed": {"$in": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}})
    elif selected_rescue == 'Tracking':
        filtered_data = shelter.read({"breed": {"$in": ["Bloodhound", "Doberman Pinscher", "German Shepherd"]}})
    else:
        filtered_data = shelter.read({})
    
    # Handle case where no data is returned
    if not filtered_data:
        data_table = []
        geo_fig = px.scatter_mapbox([], lat=[], lon=[], hover_name=[])
        additional_fig = px.pie([], names=[], title='Breed Distribution')
    else:
        # Update data table
        data_table = filtered_data
        
        # Print columns for debugging
        print("Columns in filtered data:", filtered_data[0].keys())

        # Update geolocation chart
        geo_fig = px.scatter_mapbox(
            filtered_data,
            lat="location_lat",
            lon="location_long",
            hover_name="breed",
            zoom=10
        )
        geo_fig.update_layout(mapbox_style="open-street-map")
        
        # Update additional chart (example: pie chart of breeds)
        additional_fig = px.pie(
            filtered_data,
            names='breed',
            title='Breed Distribution'
        )
    
    return data_table, geo_fig, additional_fig

# Function to run the Dash app in a separate thread
def run_dash():
    for port in range(8050, 8100):
        try:
            app.run_server(port=port, debug=False, use_reloader=False)
            break
        except OSError as e:
            if "Address already in use" in str(e):
                continue
            else:
                raise

# Run the Dash app
Thread(target=run_dash).start()


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
Dash is running on http://127.0.0.1:8051/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
Dash is running on http://127.0.0.1:8052/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8052/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Jun/2024 21:52:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2024 21:52:23] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2024 21:52:23] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2024 21:52:23] "GET /_favicon.ico?v=2.8.1 HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2024 21:52:23] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2024 21:52:23] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Jun/2024 21:52:23] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Jun/2024 21:52:23] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -


Columns in filtered data: dict_keys(['', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed', 'color', 'date_of_birth', 'datetime', 'monthyear', 'name', 'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'location_lat', 'location_long', 'age_upon_outcome_in_weeks'])


127.0.0.1 - - [23/Jun/2024 21:52:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [23/Jun/2024 21:53:49] "POST /_dash-update-component HTTP/1.1" 200 -


Columns in filtered data: dict_keys(['', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed', 'color', 'date_of_birth', 'datetime', 'monthyear', 'name', 'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'location_lat', 'location_long', 'age_upon_outcome_in_weeks'])


127.0.0.1 - - [23/Jun/2024 21:55:05] "POST /_dash-update-component HTTP/1.1" 200 -


Columns in filtered data: dict_keys(['', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed', 'color', 'date_of_birth', 'datetime', 'monthyear', 'name', 'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'location_lat', 'location_long', 'age_upon_outcome_in_weeks'])
